In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer


def question_answer(question, reference):
    """
    using bert to answer some question
    """

    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

    tokens = ['[CLS]'] + tokenizer.tokenize(question) + ['[SEP]'] + tokenizer.tokenize(reference) + ['[SEP]']
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    mask = [1] * len(input_ids)
    # apply mask,0 for reference and 1 for questions
    type_ids = [0] * (1 + len(tokenizer.tokenize(question)) + 1) + [1] * (len(tokenizer.tokenize(reference)) + 1)
    input_ids, mask, type_ids = map(lambda f: tf.expand_dims(
        tf.convert_to_tensor(f, dtype=tf.int32), 0),
                                   (input_ids, mask, type_ids))

    outputs = model([input_ids, mask, type_ids])
    

    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    if short_start <= len(tokenizer.tokenize(question)) or short_start >= short_end:
        return None

    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    return answer

2024-06-17 21:41:04.567737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 21:41:04.567863: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 21:41:04.763690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def answer_loop(reference):
    """
    Main question/answer loop.
    """
    question = ""
    quit_str = ['exit', 'quit', 'goodbye', 'bye']

    while True:
        question = input("Q: ")
        if question.lower() in quit_str:
            break
        answer = question_answer(question, reference)
        if answer is None:
            answer = "Sorry, I do not understand your question."
        print("A:", answer)

    print("A: Goodbye")

In [4]:
with open('/kaggle/input/pld-qa/PeerLearningDays.md') as f:
    reference = f.read()

answer_loop(reference)

Q:  When are PLDs?


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

A: Sorry, I do not understand your question.


Q:  What does PLD stand for?


A: peer learning days


Q:  EXIT


A: Goodbye
